In [1]:
import numpy as np
from gurobipy import *
import pandas as pd
from numpy.random import binomial, uniform
from random import sample
from itertools import combinations

In [2]:
sample_size = 200 #marketsize
products = ['A','B','C']  #A: base coverage; B:intermediate coverage; C: premium coverage
sensitivity = {'A':0.6, 'B':0.4, 'C':0.2} #people are more sensitive toward the base plan
scenarios_1 = ['s_11', 's_12', 's_13', 's_14'] #stage 1 scenarios, 4 in total
rv_1 = {'s_11':0.02, 's_12':0.5, 's_13':0.1, 's_14':0.15}
scenarios_2 = ['s_21', 's_22', 's_23'] #stage 2 scenarios, corresponds to bad, average, good. 
accident_type = ['small', 'medium', 'large'] #types of accidents
acc_rates = {'s_21':[0.45,0.3,0.1],  #bad weather
             's_22':[0.4,0.2,0.05],  #average weather
             's_23':[0.35,0.15,0.02]} #good weather

acc_cost = {'A': {'small':100, 'medium':1000, 'large':3000},
            'B': {'small':300, 'medium':1500, 'large':4000},
            'C': {'small':500, 'medium':2500, 'large':5000}}  #cost to claim each accident

prob_1 = {'s_11':0.25, 's_12':0.25,'s_13': 0.25, 's_14':0.25}
prob_2 = {'s_21':0.2, 's_22':0.6, 's_23':0.2}

price_options = {'p1':50, 'p2':100, 'p3':150, 'p4':200, 'p5':250, 'p6':300,
                 'p7':350, 'p8':400, 'p9':450, 'p10':500, 'p11':550, 'p12':600}  #all price options

price_comb = list(combinations(price_options, 3))

np.random.seed(1612)

In [3]:
def demand_func(price, marketsize, price_sense, random_var):
    dem = marketsize - price_sense*price + random_var*marketsize
    if dem <= 0:
        return 0
    if dem >= marketsize:
        return marketsize
    else:
        return dem
    
    
def demand_sim(dem, marketsize):
    prob = dem / marketsize
    return binomial(1, prob, marketsize)



def accident_sim(prob, n_customers):
    p_s, p_m, p_l = prob[0], prob[1], prob[2]
    alpha_s = binomial(3, p_s, n_customers)
    alpha_m = binomial(3, p_m, n_customers)
    alpha_l = binomial(3, p_l, n_customers)
    return alpha_s, alpha_m, alpha_l

In [4]:
demands = {}
for s1 in scenarios_1:
    for pc in price_comb:
        if (price_options[pc[0]]<price_options[pc[1]]) and (price_options[pc[1]]<price_options[pc[2]]):
            dem_A = demand_func(price_options[pc[0]], sample_size, sensitivity['A'], rv_1[s1])
            dem_B = demand_func(price_options[pc[1]], sample_size, sensitivity['B'], rv_1[s1])
            dem_C = demand_func(price_options[pc[2]], sample_size, sensitivity['C'], rv_1[s1])

            demand_A = demand_sim(dem_A, sample_size)
            demand_B = demand_sim(dem_B, sample_size)
            demand_C = demand_sim(dem_C, sample_size)
            
            dm_s1_pc = {'A': demand_A, 'B': demand_B, 'C':demand_C}
            demands[(pc,s1)] = pd.DataFrame(dm_s1_pc)

In [5]:
demands[(('p2', 'p8', 'p11'),
  's_13')]

,A,B,C
0,1,0,1
1,0,0,0
2,1,0,0
3,0,0,1
4,1,0,0
...,...,...,...
195,1,1,0
196,1,0,0
197,1,0,1
198,1,0,0


In [6]:
# simulate accidents 
accidents = {}
for s2 in scenarios_2:
    accidents_s = {}
    acc_prob = acc_rates[s2]
    accidents_s['small'], accidents_s['medium'], accidents_s['large'] = accident_sim(acc_prob, sample_size)
    accidents[s2] = pd.DataFrame(accidents_s)
        

In [7]:
# model implementation 
m = Model('OptimalPricing')

x = m.addVars(products, price_options, lb=0, ub=1, vtype=GRB.BINARY, name='x')
y = m.addVars(range(sample_size), products, scenarios_1, price_comb, vtype=GRB.BINARY, name='y')
price = m.addVars(products, lb=0, vtype=GRB.CONTINUOUS, name='p')

m.setObjective(quicksum(quicksum(prob_1[s1] * (quicksum(quicksum(price[j] * y[i,j,s1,pc[0],pc[1],pc[2]] for j in products) for i in range(sample_size)) - 
                                                                               quicksum(prob_2[s2] * (quicksum(quicksum(quicksum(accidents[s2].loc[i,a] * 
                                                                               y[i,j,s1,pc[0],pc[1],pc[2]] * acc_cost[j][a] for j in products) for i in range(sample_size)) 
                                                                              for a in accident_type)) for s2 in scenarios_2)) 
                                                                                 for s1 in scenarios_1) for pc in price_comb), GRB.MAXIMIZE)




In [8]:
# adding constraints 
m.addConstrs((price[j] == quicksum(price_options[p] * x[j,p] for p in price_options) for j in products))
m.addConstr(price['A'] <= price['B'])
m.addConstr(price['B'] <= price['C'])
m.addConstrs(((quicksum(x[j,p] for p in price_options) == 1) for j in products), 'onePriceOffer')
for s1 in scenarios_1:
    for i in range(sample_size):
        for pc in price_comb:
            m.addConstrs((y[i,j,s1,pc[0],pc[1],pc[2]] <= demands[(pc,s1)].loc[i,j] for j in products), 'demand-supply')
            m.addConstrs((3*y[i,j,s1,pc[0],pc[1],pc[2]] <= x['A',pc[0]]+x['B',pc[1]]+x['C',pc[2]] for j in products), 'offer-supply')
            m.addConstr((quicksum(y[i,j,s1,pc[0],pc[1],pc[2]] for j in products) <= 1))
        

In [9]:
m.Params.logtoconsole = 0
m.optimize()


In [10]:
x

{('A', 'p1'): <gurobi.Var x[A,p1] (value -0.0)>,
 ('A', 'p2'): <gurobi.Var x[A,p2] (value -0.0)>,
 ('A', 'p3'): <gurobi.Var x[A,p3] (value -0.0)>,
 ('A', 'p4'): <gurobi.Var x[A,p4] (value -0.0)>,
 ('A', 'p5'): <gurobi.Var x[A,p5] (value -0.0)>,
 ('A', 'p6'): <gurobi.Var x[A,p6] (value 1.0)>,
 ('A', 'p7'): <gurobi.Var x[A,p7] (value -0.0)>,
 ('A', 'p8'): <gurobi.Var x[A,p8] (value -0.0)>,
 ('A', 'p9'): <gurobi.Var x[A,p9] (value -0.0)>,
 ('A', 'p10'): <gurobi.Var x[A,p10] (value -0.0)>,
 ('A', 'p11'): <gurobi.Var x[A,p11] (value -0.0)>,
 ('A', 'p12'): <gurobi.Var x[A,p12] (value -0.0)>,
 ('B', 'p1'): <gurobi.Var x[B,p1] (value 0.0)>,
 ('B', 'p2'): <gurobi.Var x[B,p2] (value -0.0)>,
 ('B', 'p3'): <gurobi.Var x[B,p3] (value -0.0)>,
 ('B', 'p4'): <gurobi.Var x[B,p4] (value -0.0)>,
 ('B', 'p5'): <gurobi.Var x[B,p5] (value -0.0)>,
 ('B', 'p6'): <gurobi.Var x[B,p6] (value -0.0)>,
 ('B', 'p7'): <gurobi.Var x[B,p7] (value -0.0)>,
 ('B', 'p8'): <gurobi.Var x[B,p8] (value -0.0)>,
 ('B', 'p9'): <g

In [16]:

for s1 in scenarios_1:
    for i in range(sample_size):
        for j in products:
            for pc in price_comb:
                if y[i,j,s1,pc[0],pc[1],pc[2]].x >=0.5:
                    print('Provide product {} to client {}'.format(j, i))
                    print(price_options[pc[0]], price_options[pc[1]], price_options[pc[2]], s1)
#                     for a in accident_type:
#                         for s2 in scenarios_2:
#                             costs = accidents[s2].loc[i,a] * y[i,j,s1,pc[0],pc[1],pc[2]].x * acc_cost[j][a]
#                             print('Under first stage scenario {}, second stage scenario {}'.format(s1, s2))
#                             print('Costs of covering customer {}, accident type {} is {}'.format(j,a,costs))

Provide product A to client 31
300 500 550 s_11
Provide product A to client 129
300 500 550 s_11
Provide product A to client 140
300 500 550 s_11
Provide product A to client 31
300 500 550 s_12
Provide product B to client 36
300 500 550 s_12
Provide product B to client 124
300 500 550 s_12
Provide product A to client 129
300 500 550 s_12
Provide product A to client 140
300 500 550 s_12
Provide product A to client 143
300 500 550 s_12
Provide product B to client 147
300 500 550 s_12
Provide product A to client 155
300 500 550 s_12
Provide product A to client 159
300 500 550 s_12
Provide product A to client 170
300 500 550 s_12
Provide product A to client 179
300 500 550 s_12
Provide product B to client 188
300 500 550 s_12
Provide product B to client 124
300 500 550 s_13
Provide product A to client 140
300 500 550 s_13
Provide product C to client 155
300 500 550 s_13
Provide product B to client 188
300 500 550 s_13
Provide product A to client 189
300 500 550 s_13
Provide product A to cl

In [17]:
m.ObjVal

767.5

In [19]:
12**3  * 200*3*4

4147200